## Read in CSV cleaned from ntbk-1


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import csv

In [2]:
df = pd.read_csv('clean_data/2019-dems-contributions.csv', usecols=['cand_nm','contb_receipt_amt','contbr_zip','contbr_occupation','occ_cat','converted_date'])



In [3]:
df.head()

,cand_nm,contbr_zip,contbr_occupation,contb_receipt_amt,occ_cat,converted_date
0,"Sanders, Bernard",20001,UNION REPRESENTATIVE,100.0,union representative,201906
1,"Sanders, Bernard",20001,UNION REPRESENTATIVE,3.0,union representative,201906
2,"Sanders, Bernard",20001,UNION REPRESENTATIVE,27.0,union representative,201906
3,"Sanders, Bernard",20007,IT,3.0,it,201903
4,"Sanders, Bernard",20001,SOFTWARE DEVELOPER,27.0,it,201906


## Group zip codes by income 

In [4]:
print(len(list(df.contbr_zip.unique())))
print(df.contbr_zip.dtype)
df.info()

48
int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11614 entries, 0 to 11613
Data columns (total 6 columns):
cand_nm              11614 non-null object
contbr_zip           11614 non-null int64
contbr_occupation    11614 non-null object
contb_receipt_amt    11614 non-null float64
occ_cat              11614 non-null object
converted_date       11614 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 544.5+ KB


In [5]:
conn = sqlite3.connect('')
cur = conn.cursor()

In [6]:
read_income = pd.read_csv('ACS_15_5YR_S1903/ACS_15_5YR_S1903_with_ann.csv', header =0, index_col=0, usecols=['GEO.id', 'GEO.id2','HC02_EST_VC02'], skiprows=[1,2],na_values='-')


In [7]:
read_income.to_sql('INCOME', conn, if_exists = 'append')

In [8]:
df_income = pd.DataFrame(read_income)
df_income = df_income.rename(columns={'GEO.id2': 'zip', 'HC02_EST_VC02': 'income'})

In [9]:
len(df_income.income.value_counts())
df.dtypes

cand_nm               object
contbr_zip             int64
contbr_occupation     object
contb_receipt_amt    float64
occ_cat               object
converted_date         int64
dtype: object

## Take zip code and income from census table and match to contributions table

In [10]:
df_merged = df.merge(df_income, how='left', left_on='contbr_zip', right_on='zip', suffixes=('_left','_right'))

In [11]:
print(df_merged.isna().sum())
print(df_merged.dtypes)
pd.to_numeric(df_merged.income);

cand_nm                0
contbr_zip             0
contbr_occupation      0
contb_receipt_amt      0
occ_cat                0
converted_date         0
zip                  105
income               112
dtype: int64
cand_nm               object
contbr_zip             int64
contbr_occupation     object
contb_receipt_amt    float64
occ_cat               object
converted_date         int64
zip                  float64
income               float64
dtype: object


In [12]:
df_merged.shape
df.isna().sum()

cand_nm              0
contbr_zip           0
contbr_occupation    0
contb_receipt_amt    0
occ_cat              0
converted_date       0
dtype: int64

In [13]:
df_merged.groupby('cand_nm').max()

,contbr_zip,contbr_occupation,contb_receipt_amt,occ_cat,converted_date,zip,income
cand_nm,,,,,,,
"Biden, Joseph R Jr",20044,WASHINGTON DIRECTOR,2800.0,volunteer,201909,20037.0,161352.0
"Buttigieg, Pete",20052,WRITER,2800.0,wine,201909,20052.0,161352.0
"Sanders, Bernard",25413,YOGA,2719.0,yoga,201909,20540.0,161352.0
"Warren, Elizabeth",20038,WRITER/EDITOR,2800.0,writer/editor,201909,20037.0,161352.0


In [14]:
df_merged.isna().sum()

cand_nm                0
contbr_zip             0
contbr_occupation      0
contb_receipt_amt      0
occ_cat                0
converted_date         0
zip                  105
income               112
dtype: int64

In [15]:
print(df_merged.loc[df.cand_nm == 'Biden, Joseph R Jr'].mode())
print(df_merged.loc[df.cand_nm == 'Buttigieg, Pete'].mode())
print(df_merged.loc[df.cand_nm == 'Sanders, Bernard'].mode())
print(df_merged.loc[df.cand_nm == 'Warren, Elizabeth '].mode())

              cand_nm  contbr_zip contbr_occupation  contb_receipt_amt  \
0  Biden, Joseph R Jr       20016      NOT EMPLOYED               50.0   

                   occ_cat  converted_date      zip    income  
0  not employed or unknown          201909  20016.0  124080.0  
           cand_nm  contbr_zip contbr_occupation  contb_receipt_amt  \
0  Buttigieg, Pete       20009      NOT EMPLOYED              100.0   

                   occ_cat  converted_date      zip   income  
0  not employed or unknown          201909  20009.0  94213.0  
            cand_nm  contbr_zip contbr_occupation  contb_receipt_amt occ_cat  \
0  Sanders, Bernard       20009      NOT EMPLOYED               27.0     gov   

   converted_date      zip   income  
0          201909  20009.0  94213.0  
              cand_nm  contbr_zip contbr_occupation  contb_receipt_amt  \
0  Warren, Elizabeth        20009          ATTORNEY               25.0   

  occ_cat  converted_date      zip   income  
0     law          201

In [16]:
# how many income values are each candidate missing? 
print(len(df_merged.loc[(df_merged.cand_nm == 'Warren, Elizabeth ') & (df_merged.zip.isna() == True)]))
print(len(df_merged.loc[(df_merged.cand_nm == 'Biden, Joseph R Jr') & (df_merged.zip.isna() == True)]))
print(len(df_merged.loc[(df_merged.cand_nm == 'Buttigieg, Pete') & (df_merged.zip.isna() == True)]))
print(len(df_merged.loc[(df_merged.cand_nm == 'Sanders, Bernard') & (df_merged.zip.isna() == True)]))

3
2
1
99


In [17]:
# what's going on with all of Bernie Sanders' null values? 
df_bernie_nas = df_merged.loc[(df_merged.cand_nm == 'Sanders, Bernard') & (df_merged.zip.isna() == True)]
print(df_bernie_nas.contbr_zip.unique())
print(len(df_bernie_nas.contbr_zip.unique()))
df_bernie_nas.head()

[20013 20552 20212 20526 20301 22205 25413 20585 20375 20472 20270 20026
 20229 20420 20591 20500 20210 20014 20027 20577]
20


,cand_nm,contbr_zip,contbr_occupation,contb_receipt_amt,occ_cat,converted_date,zip,income
27,"Sanders, Bernard",20013,NOT EMPLOYED,3.00,not employed or unknown,201903,NaN,NaN
85,"Sanders, Bernard",20552,ECONOMIST,25.00,science,201903,NaN,NaN
92,"Sanders, Bernard",20212,ECONOMIST,3.00,science,201903,NaN,NaN
102,"Sanders, Bernard",20013,NOT EMPLOYED,16.66,not employed or unknown,201903,NaN,NaN
103,"Sanders, Bernard",20013,NOT EMPLOYED,3.00,not employed or unknown,201903,NaN,NaN


In [18]:
# Warren's missing values were for an economist so I filled that in with average salary in DC for economist 
# df_merged = df_merged.loc[(df_merged.cand_nm == 'Warren, Elizabeth ')].fillna('107000')
# df_merged.loc[(df_merged.cand_nm == 'Warren, Elizabeth ') & (df_merged.zip.isna() == True)]

In [19]:
nulls = df_merged.loc[df_merged.income.isna() == True].index
len(nulls)

112

In [20]:
# drop null income values, most of them went to Bernie Sanders 
df_merged = df_merged.drop(index=nulls)
len(df_merged)


11502

In [21]:
df_merged.head()

,cand_nm,contbr_zip,contbr_occupation,contb_receipt_amt,occ_cat,converted_date,zip,income
0,"Sanders, Bernard",20001,UNION REPRESENTATIVE,100.0,union representative,201906,20001.0,85976.0
1,"Sanders, Bernard",20001,UNION REPRESENTATIVE,3.0,union representative,201906,20001.0,85976.0
2,"Sanders, Bernard",20001,UNION REPRESENTATIVE,27.0,union representative,201906,20001.0,85976.0
3,"Sanders, Bernard",20007,IT,3.0,it,201903,20007.0,119267.0
4,"Sanders, Bernard",20001,SOFTWARE DEVELOPER,27.0,it,201906,20001.0,85976.0


In [22]:
df_merged.shape

(11502, 8)

In [23]:
df_merged.cand_nm.value_counts()

Buttigieg, Pete       3746
Warren, Elizabeth     3609
Sanders, Bernard      2726
Biden, Joseph R Jr    1421
Name: cand_nm, dtype: int64

In [26]:
df_merged.to_csv('final_data/contr-income.csv')